In [7]:
import os
from time import time
import numpy as np
import sys
sys.path.append('../')

from utils.codification_cnn import CNNLayer, NNLayer, ChromosomeCNN, FitnessCNN, FitnessCNNParallel
from utils.datamanager import DataManager
from utils.codification_skipc import ChromosomeSkip, FitnessSkip, Connections
from GA.parentSelector.parentSelector import RandomParentSelector, TournamentSelection, LinealOrder, WheelSelection
from GA.geneticAlgorithm import GenerationalGA


In [8]:
os.cpu_count()
os.name

'posix'

In [9]:
# Chromosome parameters
mutation_rate = 0.05
mutation_rate_1 = mutation_rate * 2
ChromosomeSkip.grow_prob = mutation_rate
ChromosomeSkip.decrease_prob = mutation_rate

CNNLayer.k_prob = mutation_rate
CNNLayer.filter_prob = mutation_rate_1
CNNLayer.act_prob = mutation_rate
CNNLayer.drop_prob = mutation_rate_1
CNNLayer.maxpool_prob = mutation_rate

NNLayer.units_prob = mutation_rate_1
NNLayer.act_prob = mutation_rate
NNLayer.drop_prob = mutation_rate_1

CNNLayer.filters_lim = 128


if os.cpu_count() == 8:
    data_folder = '/home/daniel/datasets/MNIST_variations'
    command = 'python ./../train_gen.py'
    verbose = 1
    experiments_folder = '../exps4'
    gpus = 1
    epochs = 2
    precise_eps = 5
    num_parents = 0.25 * 2
    generations = 2
    population = 4
    precision_individuals = 1
else:
    data_folder = '../../datasets'
    command = 'python3 ./train_gen.py'
    verbose = 0
    experiments_folder = '../../exps'
    gpus = 2
    epochs = 15
    precise_eps = 75
    num_parents = 0.25
    generations = 30
    population = 20
    precision_individuals = 3

skip = True

In [11]:
experiments_folder = experiments_folder
os.makedirs(experiments_folder, exist_ok=True)

if skip:
    fitness_cnn = FitnessSkip()
else:
    fitness_cnn = FitnessCNN()

l = CNNLayer(32, (3,3), 'relu', 0, False)
l2 = CNNLayer(64, (3,3), 'relu', 0.25, True)
l3 = NNLayer(128, 'relu', 0.5)
c = ChromosomeCNN([l, l2], [l3], fitness_cnn)
if skip:
    c = ChromosomeSkip(c.cnn_layers, c.nn_layers, Connections.random_connections(len(c.cnn_layers)))
    
# dataset params:
data_folder = data_folder
classes = []

# genetic algorithm params:
parents_selector_key = 'tournament'
num_parents = num_parents
generations = generations
population = population
train_time = 10
maximize_fit = False
stat_val = True
save_progress = True
age_survivors_rate = 0.05
precision_val = True
precision_individuals = precision_individuals

ps = {'random':RandomParentSelector(), 'lineal':LinealOrder(), 'wheel':WheelSelection(), 
      'tournament':TournamentSelection(population//4)}
p = ps[parents_selector_key]

# Fitness params
epochs = epochs
batch_size = 128
verbose = verbose
redu_plat = False
early_stop = 15
warm_up_epochs= 0
base_lr = 0.001
smooth = 0.1
cosine_dec = True
lr_find = False
precise_eps = precise_eps

datasets = ['MB', 'MBI', 'MRB', 'MRD', 'MRDBI']
#datasets = ['MRDBI']

In [13]:
for dataset in datasets:
    print("\nEVOLVING IN DATASET %s ...\n" % dataset)
    exp_folder = os.path.join(experiments_folder, dataset)
    folder = os.path.join(exp_folder, 'genetic')
    fitness_folder = exp_folder
    fitness_file = os.path.join(fitness_folder, 'fitness_example') 
    try:
        generational = GenerationalGA.load_genetic_algorithm(folder=folder)
    except FileNotFoundError:
        os.makedirs(folder, exist_ok=True)

        # Load data
        dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
        data = dm.load_data()
        fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                       epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                       warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find,
                       precise_epochs=precise_eps)

        fitness_cnn.save(fitness_file)

        del dm, data

        fitness = FitnessCNNParallel()
        fitness.set_params(chrom_files_folder=fitness_folder, fitness_file=fitness_file, max_gpus=gpus,
                       fp=32, main_line=command)

        generational = GenerationalGA(fitness=fitness, num_parents=num_parents, chromosome=c, parent_selector=p,
                                  generations=generations, num_population=population, maximize_fitness=maximize_fit, 
                                  training_hours=train_time, folder=folder, statistical_validation=stat_val,
                                  save_progress=save_progress, age_survivors_rate=age_survivors_rate,
                                  precision_val=precision_val, precision_individuals=precision_individuals)

    ti_all = time()
    winner, best_fit, ranking = generational.evolve()
    print("Total elapsed time: %0.3f" % (time() - ti_all))


EVOLVING IN DATASET MB ...



KeyboardInterrupt: 